In [1]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import torch
import pickle
import glob
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas
import time
import os

import connect_four
import mcts
import connect_net
import connect_plot
import evaluation
import utils
import runner
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
os.chdir("/Users/raymond/code/FinalProject563")
trial = "test01"
iteration = 5
training_folder = utils.get_training_folder(trial=trial, iteration=iteration)
training_path = runner.get_model_path(folder=training_folder, iteration=iteration)
nnet8 = connect_net.load_model(path=training_path)
iteration = 1 
training_folder = utils.get_training_folder(trial=trial, iteration=iteration)
training_path = runner.get_model_path(folder=training_folder, iteration=iteration)
nnet1 = connect_net.load_model(path=training_path)
nnetp = connect_net.load_model(path="models/pretrain01.model")

ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): Tanh()
  )
  (fc): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 4), stride=(1, 1))
    (1): Tanh()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Tanh()
    (5): Linear(in_features=128, out_features=8, bias=True)
  )
)
ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): Tanh()
  )
  (fc): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 4), stride=(1, 1))
    (1): Tanh()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Tanh()
    (5): Linear(in_features=128, out_features=8, bias=True)
  )
)
ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): Tanh()
  )
  (fc): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 4), stride=(1, 1))
    (1):

In [7]:
num_mcts_sims = 32
results = []
for i in range(100):
    result = evaluation.competition_game(
        player1=nnet1, p1_mcts_sims=num_mcts_sims, 
        player2=nnet8, p2_mcts_sims=num_mcts_sims,
    )
    results.append(result)

In [8]:
df_results = pandas.DataFrame([(r[0], r[1]) for r in results], columns=["Winner", "moves"])
df_results["Winner"].value_counts()

player2    59
player1    33
Draw        8
Name: Winner, dtype: int64